# pyDatalog使用

In [1]:
from pyDatalog import pyDatalog
pyDatalog.create_terms('factorial, N')
factorial[N] = N*factorial[N-1] 
factorial[1] = 1 
print(factorial[3]==N)  

N
-
6


In [2]:
# 一个小推理规则
pyDatalog.create_terms('X,Y,Z,father,fatherOf,grandfatherOf')
(grandfatherOf[X] == Z) <= ((fatherOf[X]==Y) & (fatherOf[Y]==Z))
fatherOf["乾隆"] = "雍正"
fatherOf["雍正"] = "康熙"
print(grandfatherOf["乾隆"] == X)

X 
--
康熙


## 变量与表达式

In [3]:
print(X==1)

X
-
1


In [4]:
print((X==True) & (Y==False))

X    | Y    
-----|------
True | False


In [5]:
print((X==True) & (X==False))

[]


In [6]:
print((X==input('Please enter your name : ')) & (Y=='Hello ' + X[0]))

Please enter your name : haidfs
X      | Y      
-------|--------
haidfs | Hello h


In [7]:
#在第二个等式中，X被称为受第一个等式的约束，也就是说要在第一个等式给X一个值，才使得第二个等式中关于X的表达式（Y）有可能被估值。
print((Y==1) & (Y==X+1))

[]


In [8]:
#变量也可以表示（嵌套的）元组，它们可以参与表达式并被切片（0为基）。
print((X==(1,2)+(3,)) & (Y==X[2]))

X         | Y
----------|--
(1, 2, 3) | 3


In [9]:
#要在逻辑表达式中使用自己的函数，请在Python中定义它们，然后在pyDatalog为它们创建逻辑术语：
def twice(a):
    return a+a

pyDatalog.create_terms('twice')
print((X==1) & (Y==twice(X)))

X | Y
--|--
1 | 2


In [10]:
#同样，pyDatalog变量可以传递给Python标准库中的函数：
import math
pyDatalog.create_terms('math')
print((X==2) & (Y==math.sqrt(X)))

X | Y                 
--|-------------------
2 | 1.4142135623730951


In [11]:
#循环可以通过使用.in_() 方法创建 （我们将在以后看到还有其他方法可以创建循环）： 
#【注：这里没有使用==，但同样执行了一次查询，查询的结果存在X中】
pyDatalog.create_terms('X,Y,Z')
print(X.in_((0,1,2,3,4)))

X
-
4
3
2
1
0


In [12]:
print(X.in_(range(5)).data)
print(X.in_(range(5)) == set([(0,), (1,), (2,), (3,), (4,)]))

[(4,), (3,), (2,), (1,), (0,)]
True


In [13]:
print("Data : ",X.data)
print("First value : ",  X.v())
# below, '>=' is a variable extraction operator
print("Extraction of first value of X: ", X.in_(range(5)) >= X)

Data :  [4, 3, 2, 1, 0]
First value :  4
Extraction of first value of X:  4


In [14]:
print(X.in_(range(5))& (X<2))

X
-
1
0


In [15]:
#循环嵌套，使用缩进以提高可读性
print(X.in_(range(5)) &
          Y.in_(range(5)) &
              (Z==X+Y) &
              (Z<3))

X | Y | Z
--|---|--
2 | 0 | 2
1 | 1 | 2
1 | 0 | 1
0 | 2 | 2
0 | 1 | 1
0 | 0 | 0


## 逻辑函数与字典

In [16]:
pyDatalog.create_terms('X,Y,Z, salary, tax_rate, tax_rate_for_salary_above, net_salary')
salary['foo'] = 60
salary['bar'] = 110
print(salary[X]==Y)
print({X.data[i]:Y.data[i] for i in range(len(X.data))})        #【真正转化为字典的写法】

X   | Y  
----|----
bar | 110
foo | 60 
{'bar': 110, 'foo': 60}


In [17]:
salary['foo'] = 60
salary['bar'] = 110

# Python equivalent
_salary = dict()
_salary['foo'] = 60
_salary['bar'] = 110
# give me all the X and Y so that the salary of X is Y
print(salary[X]==Y)
print
# python equivalent
print(_salary.items())

X   | Y  
----|----
bar | 110
foo | 60 
dict_items([('foo', 60), ('bar', 110)])


In [18]:
#一个函数对于给定的参数具备唯一值，
salary['foo'] = 70
print(salary['foo']==Y)
print

Y 
--
70


<function print>

In [19]:
# Python equivalent
_salary['foo'] = 70
print('foo --> ' + str(_salary['foo']))

foo --> 70


In [20]:
#一个函数也可以使用键来进行查询
print(salary[X]==110)
#查询可以测试一个标准的否定
print((salary[X]==Y) & ~(Y==110))

X  
---
bar
X   | Y 
----|---
foo | 70


In [21]:
#定义全球税率
+(tax_rate[None]==0.33)
# 一个函数可以在公式中调用：
# give me the net salary for all X
print((Z==salary[X]*(1-tax_rate[None])))

X   | Z                
----|------------------
bar | 73.69999999999999
foo | 46.89999999999999


In [22]:
#一个函数可以由一个子句定义，以下是一个简单的例子
# the net salary of X is Y if Y is the salary of X, reduced by the tax rate
net_salary[X] = salary[X]*(1-tax_rate[None])
print(net_salary[X]==Y)

X   | Y                
----|------------------
bar | 73.69999999999999
foo | 46.89999999999999


In [23]:
(tax_rate_for_salary_above[X] == 0.33) <= (0 <= X)
(tax_rate_for_salary_above[X] == 0.50) <= (100 <= X)
print(tax_rate_for_salary_above[70]==Y)
print(tax_rate_for_salary_above[150]==Y)

Y   
----
0.33
Y  
---
0.5


## 这里第一次出现了推理

In [24]:
#"<="是上述陈述中的重要标志：它被读作'if'。【可以用来定义“推出”的规则】
#请注意，我们在上面使用的f[X]=，是(f[X]==Y) <= (Y==expr)的简写。
#这个简短的表示法以及可以按任意顺序定义函数的事实，使得编写pyDatalog程序像创建电子表格一样简单。
del net_salary[X]
net_salary[X] = salary[X]*(1-tax_rate_for_salary_above[salary[X]])
print(net_salary[X]==Y)

X   | Y                
----|------------------
bar | 55.0             
foo | 46.89999999999999


## 聚合函数

In [25]:
from pyDatalog import pyDatalog
pyDatalog.create_terms('X,Y,manager, count_of_direct_reports')
# the manager of Mary is John
+(manager['Mary'] == 'John')
+(manager['Sam']  == 'Mary')
+(manager['Tom']  == 'Mary')

## 字面值与集合

In [26]:
from pyDatalog import pyDatalog
pyDatalog.create_terms('X,Y,Z, works_in, department_size, manager, indirect_manager, count_of_indirect_reports')
# Mary works in Production
+ works_in('Mary', 'Production')
+ works_in('Sam',  'Marketing')

In [27]:
+ works_in('John', 'Production')
+ works_in('John', 'Marketing')
# give me all the X that work in Marketing
print(works_in(X,  'Marketing'))

X   
----
John
Sam 


In [28]:
# one of the indirect manager of X is Y, if the (direct) manager of X is Y
indirect_manager(X,Y) <= (manager[X] == Y)
# another indirect manager of X is Y, if there is a Z so that the manager of X is Z, 
#   and an indirect manager of Z is Y
indirect_manager(X,Y) <= (manager[X] == Z) & indirect_manager(Z,Y)
print(indirect_manager('Sam',X))

X   
----
Mary
John


## 树、图与递归算法

In [29]:
pyDatalog.create_terms('link, can_reach')
# there is a link between node 1 and node 2
+link(1,2)
+link(2,3)
+link(2,4)
+link(2,5)
+link(5,6)
+link(6,7)
+link(7,2)

In [30]:
# 无向图，边双向连接
link(X,Y) <= link(Y,X)

link(X,Y) <= link(Y,X)

In [31]:
# can Y be reached from X ?
can_reach(X,Y) <= link(X,Y) # direct link
# via Z
can_reach(X,Y) <= link(X,Z) & can_reach(Z,Y) & (X!=Y)

print (can_reach(1,Y))

Y
-
2
6
7
4
3
5


## 八皇后问题 

In [32]:
from pyDatalog import pyDatalog
pyDatalog.create_terms('N,X0,X1,X2,X3,X4,X5,X6,X7')
pyDatalog.create_terms('ok,queens,next_queen')

# the queen in the first column can be in any row
queens(X0)                      <= (X0._in(range(8)))

# to find the queens in the first 2 columns, find the first one first, then find a second one
queens(X0,X1)                   <= queens(X0)                   & next_queen(X0,X1)

# repeat for the following queens
queens(X0,X1,X2)                <= queens(X0,X1)                & next_queen(X0,X1,X2)
queens(X0,X1,X2,X3)             <= queens(X0,X1,X2)             & next_queen(X0,X1,X2,X3)
queens(X0,X1,X2,X3,X4)          <= queens(X0,X1,X2,X3)          & next_queen(X0,X1,X2,X3,X4)
queens(X0,X1,X2,X3,X4,X5)       <= queens(X0,X1,X2,X3,X4)       & next_queen(X0,X1,X2,X3,X4,X5)
queens(X0,X1,X2,X3,X4,X5,X6)    <= queens(X0,X1,X2,X3,X4,X5)    & next_queen(X0,X1,X2,X3,X4,X5,X6)
queens(X0,X1,X2,X3,X4,X5,X6,X7) <= queens(X0,X1,X2,X3,X4,X5,X6) & next_queen(X0,X1,X2,X3,X4,X5,X6,X7)

# the second queen can be in any row, provided it is compatible with the first one
next_queen(X0,X1)                   <= queens(X1)                       & ok(X0,1,X1)

# to find the third queen, first find a queen compatible with the second one, then with the first
# re-use the previous clause for maximum speed, thanks to memoization
next_queen(X0,X1,X2)                <= next_queen(X1,X2)                & ok(X0,2,X2)

# repeat for all queens
next_queen(X0,X1,X2,X3)             <= next_queen(X1,X2,X3)             & ok(X0,3,X3)
next_queen(X0,X1,X2,X3,X4)          <= next_queen(X1,X2,X3,X4)          & ok(X0,4,X4)
next_queen(X0,X1,X2,X3,X4,X5)       <= next_queen(X1,X2,X3,X4,X5)       & ok(X0,5,X5)
next_queen(X0,X1,X2,X3,X4,X5,X6)    <= next_queen(X1,X2,X3,X4,X5,X6)    & ok(X0,6,X6)
next_queen(X0,X1,X2,X3,X4,X5,X6,X7) <= next_queen(X1,X2,X3,X4,X5,X6,X7) & ok(X0,7,X7)

# it's ok to have one queen in row X1 and another in row X2 if they are separated by N columns
ok(X1, N, X2) <= (X1 != X2) & (X1 != X2+N) & (X1 != X2-N)

ok(X1,N,X2) <= !=(X1,X2)&!=(X1,(X2+N))&!=(X1,(X2-N

In [33]:
# give me one solution to the 8-queen puzzle
print(queens(X0,X1,X2,X3,X4,X5,X6,X7).data[0])

(7, 3, 0, 2, 5, 1, 6, 4)
